In [ ]:
import phoenix as px

pxc = px.Client()

In [ ]:
from phoenix.session.evaluation import get_retrieved_documents

retrieved_documents = get_retrieved_documents(pxc, project_name="research_assistant")
retrieved_documents.head()

In [ ]:
import nest_asyncio

from phoenix.evals import (
    OpenAIModel,
    llm_classify,
)

nest_asyncio.apply()

model = OpenAIModel(
    model="gpt-4o",
    temperature=0.0,
)

bias_detection_prompt = """
You are an AI assistant designed to detect bias in documents. Given an original question
and a document, your task is to analyze the document and determine if there is any bias present.

Please follow these steps:
1. Read the original question carefully.
2. Read the document provided.
3. Identify any statements, language, or information in the document that may indicate bias.
4. Provide a classification of the document's bias level:
Unbiased, Somewhat Unbiased, Somewhat Biased, Biased.
5. Provide a detailed explanation for your classification, citing specific parts of the
document that influenced your decision.

Original Question: {input}
Document: {reference}

Your analysis should be thorough and objective. Please ensure that your explanation
is clear and concise.

Example response:
************
EXPLANATION: An explanation of your reasoning for the label you chose
LABEL: "bias", "unbiased", "somewhat biased", "somewhat unbiased"
************
"""

In [ ]:
bias_classifications = llm_classify(
    dataframe=retrieved_documents,
    template=bias_detection_prompt,
    model=model,
    rails=["Unbiased", "Biased", "Somewhat Biased", "Somewhat Unbiased"],
    provide_explanation=True,
)
bias_classifications["score"] = bias_classifications["label"].map(
    {"unbiased": 1, "somewhat unbiased": 0.75, "somewhat biased": 0.5, "biased": 0}
)

In [ ]:
bias_classifications.head()

In [ ]:
span_bias_classifications = bias_classifications.copy()

span_bias_classifications["average_score"] = span_bias_classifications.groupby("context.span_id")[
    "score"
].transform("mean")

span_bias_classifications["label"] = (
    span_bias_classifications["average_score"]
    .apply(
        lambda x: min(
            {
                1: "unbiased",
                0.75: "somewhat unbiased",
                0.5: "somewhat biased",
                0: "biased",
            }.keys(),
            key=lambda k: abs(k - x),
        )
    )
    .map({1: "unbiased", 0.75: "somewhat unbiased", 0.5: "somewhat biased", 0: "biased"})
)

# Combine all rows with the same context.span_id into one row, with explanations being a concatenation of all the explanations
span_bias_classifications = (
    span_bias_classifications.groupby("context.span_id")
    .agg(
        {
            "label": "first",
            "explanation": lambda x: "\n----\n".join(x),
            "exceptions": "first",
            "execution_status": "first",
            "execution_seconds": "mean",
            "score": "mean",
            "average_score": "first",
        }
    )
    .reset_index()
)
span_bias_classifications.set_index("context.span_id", inplace=True)
span_bias_classifications.head()

In [ ]:
from phoenix.trace import DocumentEvaluations, SpanEvaluations

px.Client().log_evaluations(
    SpanEvaluations(
        dataframe=span_bias_classifications,
        eval_name="Bias Detection",
    ),
    DocumentEvaluations(
        dataframe=bias_classifications,
        eval_name="Relevance",
    ),
)